Expand identity term lexicon to terms specific to incels data

# Train Gensim word2vec model from pretrained embeddings
Never got working with Gensim 4

In [ ]:
import gensim.downloader

vectors = gensim.downloader.load('word2vec-google-news-300')
type(vectors)

In [2]:
type(vectors)

gensim.models.keyedvectors.KeyedVectors

# Get nearest neighbors of identity terms in embedding space
Get top words above a frequency threshold for every individual identity term  
Also look into nearest neighbors of centroids of all terms in identity space

In [21]:
from gensim.models import Word2Vec

path = '../models/emb/incels.model'
model = Word2Vec.load(path)

In [22]:
model.wv.key_to_index

{'.': 0,
 ',': 1,
 'i': 2,
 'the': 3,
 'to': 4,
 'a': 5,
 'and': 6,
 ':': 7,
 'you': 8,
 'said': 9,
 'of': 10,
 'is': 11,
 'it': 12,
 'that': 13,
 'in': 14,
 'are': 15,
 'for': 16,
 "n't": 17,
 'they': 18,
 "'s": 19,
 'do': 20,
 '?': 21,
 '"': 22,
 'this': 23,
 'have': 24,
 'be': 25,
 'with': 26,
 'but': 27,
 'not': 28,
 'if': 29,
 'my': 30,
 'on': 31,
 'like': 32,
 'just': 33,
 'me': 34,
 'was': 35,
 'as': 36,
 'he': 37,
 'or': 38,
 'so': 39,
 'at': 40,
 'your': 41,
 'would': 42,
 'all': 43,
 '-': 44,
 'get': 45,
 'no': 46,
 'what': 47,
 'can': 48,
 'will': 49,
 'about': 50,
 'even': 51,
 ')': 52,
 'she': 53,
 'because': 54,
 'an': 55,
 'who': 56,
 'them': 57,
 '(': 58,
 'nt': 59,
 'their': 60,
 'there': 61,
 'we': 62,
 'women': 63,
 'people': 64,
 'her': 65,
 'when': 66,
 'from': 67,
 'by': 68,
 'how': 69,
 'only': 70,
 'more': 71,
 'one': 72,
 'some': 73,
 'why': 74,
 "'m": 75,
 'up': 76,
 'out': 77,
 "'re": 78,
 'being': 79,
 'think': 80,
 '!': 81,
 'good': 82,
 'than': 83,
 'fuck'

In [23]:
# Load identity terms present 

# Load identity terms
import json
import pandas as pd

identities_path = '../resources/generic_agents-identity_v15_2021_10_15.xlsx'
identities_include_path = '../resources/netmapper_include.json'
identities_exclude_path = '../resources/netmapper_exclude.json'
multi_identities = pd.read_excel(identities_path)
with open(identities_exclude_path) as f:
    identities_exclude = json.load(f)
with open(identities_include_path) as f:
    identities_include = json.load(f)

# Filter to English, remove duplicates
cols = multi_identities.columns.tolist()
en_identities = multi_identities[cols[cols.index('English'):]].copy()
en_identities['term'] = en_identities['English'].str.lower()
en_identities.drop_duplicates(subset='term', inplace=True)

# Separate out stopwords
stops = en_identities[en_identities['stop word']==1]
identities = en_identities[
    (en_identities['stop word']!=1) & (~en_identities['term'].isin(identities_exclude))
]['term'].tolist() + identities_include

vocab_path = '../tmp/incels_vocab.json'
with open(vocab_path, 'r') as f:
    vocab = json.load(f)
print(f'\t\tLoaded vocab from {vocab_path}')

present_identities = [term for term in identities if term in vocab]
print(f'\t\t{len(present_identities)} out of {len(identities)} identity terms present in vocab')
identities = present_identities

		Loaded vocab from ../tmp/incels_vocab.json
		7104 out of 18909 identity terms present in vocab


In [24]:
# Load data with identity term matches extracted
import pandas as pd

path = '../../data/incels/processed_comments.pkl'
# path = '../data/white_supremacist_identities.pkl'
data = pd.read_pickle(path)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6248230 entries, 0 to 6248229
Data columns (total 9 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   type                        object        
 1   forum                       object        
 2   thread                      object        
 3   username                    object        
 4   date                        object        
 5   content                     object        
 6   parsed_date                 datetime64[ns]
 7   content_orig                object        
 8   netmapper_identity_matches  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 429.0+ MB


In [30]:
# Get frequencies of identity terms
from collections import Counter

exp = data.explode('netmapper_identity_matches')
freq = exp.netmapper_identity_matches.value_counts()
freq.head(20)

women       546358
men         394203
man         252608
girls       175850
girl        161699
bro         150707
woman       132928
female      131720
male        121245
females      85511
virgin       74810
dude         68028
males        65007
gay          64381
kids         59433
whore        55064
whores       53916
asian        53560
bitch        50786
subhuman     50528
Name: netmapper_identity_matches, dtype: int64

In [40]:
# Build frequency dictionary for all vocabulary
model.wv.get_vecattr('women', 'count')

569416

In [42]:
# Get nearest neighbors for individual terms
from pprint import pprint

freq_threshold = 1000

for term in freq.head(30).index:
    print(term)
    pprint([(word, count) for word, count in model.wv.most_similar(term, topn=30) if word not in identities and model.wv.get_vecattr(word, 'count') >= 100])
    print()

women
[('foids', 0.8714874386787415),
 ('femoids', 0.8624152541160583),
 ('roasties', 0.6289045214653015),
 ('people', 0.602886438369751),
 ('womyn', 0.5922061800956726),
 ('noodlewhores', 0.5912578105926514),
 ('holes', 0.5884185433387756),
 ('chicks', 0.5878169536590576),
 ('fhos', 0.5734006762504578),
 ('wahmen', 0.5392026901245117),
 ('landwhales', 0.5146744847297668),
 ('w*men', 0.5113608837127686),
 ('jbs', 0.5096724033355713),
 ('femails', 0.5087173581123352),
 ('them', 0.49983498454093933),
 ('jailbaits', 0.4991641938686371),
 ('they', 0.4989631175994873),
 ('fems', 0.49879807233810425)]

men
[('femoids', 0.6556289792060852),
 ('guys', 0.6545674800872803),
 ('foids', 0.6525324583053589),
 ('people', 0.651522159576416),
 ('betas', 0.5970030426979065),
 ('incels', 0.5918691754341125),
 ('ethnics', 0.58066725730896),
 ('chads', 0.559548556804657),
 ('minorities', 0.5339909791946411),
 ('ones', 0.531589686870575),
 ('subhumans', 0.523898720741272),
 ('races', 0.5191388130187988),
 

In [69]:
# Get nearest neighbors for identity groups
import json
from pprint import pprint
from collections import defaultdict

identity_groups_fpath = '../resources/identity_groups.json'
with open(identity_groups_fpath, 'r') as f:
    identity_groups = json.load(f)
print(len(identity_groups))

gp_vals = defaultdict(list)
for key, vals in identity_groups.items():
    for gp in vals:
        gp_vals[gp].append(key)
        
freq_threshold = 1000

for gp, terms in gp_vals.items():
    print(gp)
    filtered_terms = [term for term in terms if term in model.wv.key_to_index]
    pprint([(word, count) for word, count in model.wv.most_similar(filtered_terms, topn=30) if word not in identities and model.wv.get_vecattr(word, 'count') >= 100])
    print()

213
women_girls
[('foid', 0.7099194526672363),
 ('roastie', 0.648100733757019),
 ('femoid', 0.6422341465950012),
 ('foids', 0.6307872533798218),
 ('roasties', 0.6272706389427185),
 ('femoids', 0.5961098074913025),
 ('stacy', 0.5911198854446411),
 ('chick', 0.5739116072654724),
 ('landwhale', 0.5735090970993042),
 ('chicks', 0.5590694546699524),
 ('noodlewhore', 0.5539418458938599),
 ('holes', 0.5400814414024353),
 ('skanks', 0.53519606590271),
 ('thots', 0.531789243221283),
 ('milf', 0.531764030456543),
 ('stacey', 0.5277373790740967),
 ('cumdumpster', 0.5162137150764465),
 ('mistresses', 0.5112111568450928),
 ('jb', 0.508441686630249),
 ('landwhales', 0.5080294609069824),
 ('thot', 0.5051518082618713),
 ('jbs', 0.5012690424919128),
 ('stacies', 0.4992675185203552),
 ('hoe', 0.4988044202327728)]

men_boys
[('dad', 0.5996987223625183),
 ('guy', 0.5867642164230347),
 ('bfs', 0.5861440896987915),
 ('guys', 0.558027446269989),
 ('mistresses', 0.5230361223220825),
 ('cousin', 0.517040312290

In [70]:
# Look into use of these terms
pd.set_option('display.max_colwidth', None)

term = 'slag'
data[data.content.str.contains(term)].sample(10).content

914958                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  